In [1]:
from mindspore import context
from mindspore.nn import Adam
from sponge import Sponge
from sponge import Molecule
from sponge import ForceField
from sponge import UpdaterMD
from sponge import WithEnergyCell
from sponge.potential import SphericalRestrict
from sponge.function import VelocityGenerator
from sponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
mini = Sponge(system, potential, opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [8]:
run_info = RunInfo(10)
mini.run(1000, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-06-30 16:27:08
[MindSPONGE] Step: 0, E_pot: 242.99136
[MindSPONGE] Step: 10, E_pot: 162.66307
[MindSPONGE] Step: 20, E_pot: 91.999916
[MindSPONGE] Step: 30, E_pot: 21.866974
[MindSPONGE] Step: 40, E_pot: -43.896427
[MindSPONGE] Step: 50, E_pot: -100.5366
[MindSPONGE] Step: 60, E_pot: -159.18906
[MindSPONGE] Step: 70, E_pot: -217.35576
[MindSPONGE] Step: 80, E_pot: -263.58884
[MindSPONGE] Step: 90, E_pot: -299.75137
[MindSPONGE] Step: 100, E_pot: -330.5068
[MindSPONGE] Step: 110, E_pot: -359.63373
[MindSPONGE] Step: 120, E_pot: -389.3228
[MindSPONGE] Step: 130, E_pot: -417.74637
[MindSPONGE] Step: 140, E_pot: -441.52087
[MindSPONGE] Step: 150, E_pot: -459.2307
[MindSPONGE] Step: 160, E_pot: -471.3694
[MindSPONGE] Step: 170, E_pot: -479.86255
[MindSPONGE] Step: 180, E_pot: -486.56183
[MindSPONGE] Step: 190, E_pot: -492.54034
[MindSPONGE] Step: 200, E_pot: -498.02493
[MindSPONGE] Step: 210, E_pot: -502.83423
[MindSPONGE] Step: 220, E_pot: -506.86633

In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.shape, system.atom_mass)

In [10]:
opt = UpdaterMD(system=system,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',)

## 外加限制

在MindSPONGE的WithEnergyCell中可以自定义一些外加的势能项，这是一个非常灵活的操作，支持同时定义多个Bias叠加使用，并且在MindSpore框架的支持下都是可微的。比如本案例中施加的Bias是中心球形势SphericalRestrict，用于把原子的运动限制在一个球形范围内。另外MindSponge也支持原位谐振子势OscillatorBias，可以将原子限制在原始位置附近，这个功能在蛋白质预测中能够起到较大的作用。基于MindSPONGE的基础类PotentialCell，用户还可以自定义更多种类的Bias。

In [11]:
sim = WithEnergyCell(system, potential, bias=SphericalRestrict(radius=1.5, center=[0, 0, 0]))
md = Sponge(sim, optimizer=opt)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b04.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.run(2000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-06-30 16:27:22
[MindSPONGE] Step: 0, E_pot: -546.09375, E_kin: 163.0075, E_tot: -383.08624, Temperature: 373.43423
[MindSPONGE] Step: 10, E_pot: -485.7534, E_kin: 112.67149, E_tot: -373.0819, Temperature: 258.11935
[MindSPONGE] Step: 20, E_pot: -464.64957, E_kin: 92.389885, E_tot: -372.25967, Temperature: 211.65617
[MindSPONGE] Step: 30, E_pot: -454.0137, E_kin: 76.92503, E_tot: -377.08868, Temperature: 176.2277
[MindSPONGE] Step: 40, E_pot: -469.77313, E_kin: 86.45755, E_tot: -383.31558, Temperature: 198.06578
[MindSPONGE] Step: 50, E_pot: -465.74377, E_kin: 99.71851, E_tot: -366.02527, Temperature: 228.44534
[MindSPONGE] Step: 60, E_pot: -454.39484, E_kin: 94.81408, E_tot: -359.58075, Temperature: 217.20978
[MindSPONGE] Step: 70, E_pot: -443.4588, E_kin: 95.24145, E_tot: -348.21735, Temperature: 218.18883
[MindSPONGE] Step: 80, E_pot: -443.7237, E_kin: 103.214676, E_tot: -340.50903, Temperature: 236.45473
[MindSPONGE] Step: 90, E_pot: -443.0663